# SQL notes

Author: Matthew K. MacLeod

### Tutorial goals:

* create some databases
* loading data
* some advanced topics
    * normalization
    * theta joins
    * projection
    * subqueries
    * views
    * analytics

## Data

getting some data from 538 on github

https://github.com/fivethirtyeight/data/tree/master/college-majors


background:



## Explore data sets

In [64]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
os.getcwd()
df = pd.read_csv('data/all-ages.csv')

In [5]:
df.columns

Index(['Major_code', 'Major', 'Major_category', 'Total', 'Employed',
       'Employed_full_time_year_round', 'Unemployed', 'Unemployment_rate',
       'Median', 'P25th', 'P75th'],
      dtype='object')

In [6]:
df.describe()

,Major_code,Total,Employed,Employed_full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th
count,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000
mean,3879.815029,230256.635838,166161.982659,126307.774566,9725.034682,0.057355,56816.184971,38697.109827,82506.358382
std,1687.753140,422068.483149,307324.399668,242425.389073,18022.040192,0.019177,14706.226865,9414.524761,20805.330126
min,1100.000000,2396.000000,1492.000000,1093.000000,0.000000,0.000000,35000.000000,24900.000000,45800.000000
25%,2403.000000,24280.000000,17281.000000,12722.000000,1101.000000,0.046261,46000.000000,32000.000000,70000.000000
50%,3608.000000,75791.000000,56564.000000,39613.000000,3619.000000,0.054719,53000.000000,36000.000000,80000.000000
75%,5503.000000,205763.000000,142879.000000,111025.000000,8862.000000,0.069043,65000.000000,42000.000000,95000.000000
max,6403.000000,3123510.000000,2354398.000000,1939384.000000,147261.000000,0.156147,125000.000000,78000.000000,210000.000000


In [7]:
df.head()

,Major_code,Major,Major_category,Total,Employed,Employed_full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th
0,1100,GENERAL AGRICULTURE,Agriculture & Natural Resources,128148,90245,74078,2423,0.026147,50000,34000,80000
1,1101,AGRICULTURE PRODUCTION AND MANAGEMENT,Agriculture & Natural Resources,95326,76865,64240,2266,0.028636,54000,36000,80000
2,1102,AGRICULTURAL ECONOMICS,Agriculture & Natural Resources,33955,26321,22810,821,0.030248,63000,40000,98000
3,1103,ANIMAL SCIENCES,Agriculture & Natural Resources,103549,81177,64937,3619,0.042679,46000,30000,72000
4,1104,FOOD SCIENCE,Agriculture & Natural Resources,24280,17281,12722,894,0.049188,62000,38500,90000


In [14]:
df.tail()

,Major_code,Major,Major_category,Total,Employed,Employed_full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th
168,6211,HOSPITALITY MANAGEMENT,Business,200854,163393,122499,8862,0.051447,49000,33000,70000
169,6212,MANAGEMENT INFORMATION SYSTEMS AND STATISTICS,Business,156673,134478,118249,6186,0.043977,72000,50000,100000
170,6299,MISCELLANEOUS BUSINESS & MEDICAL ADMINISTRATION,Business,102753,77471,61603,4308,0.052679,53000,36000,83000
171,6402,HISTORY,Humanities & Liberal Arts,712509,478416,354163,33725,0.065851,50000,35000,80000
172,6403,UNITED STATES HISTORY,Humanities & Liberal Arts,17746,11887,8204,943,0.073500,50000,39000,81000


In [65]:
df_rec = pd.read_csv('data/recent-grads.csv')

In [66]:
df_rec.columns

Index(['Rank', 'Major_code', 'Major', 'Major_category', 'Total', 'Sample_size',
       'Men', 'Women', 'ShareWomen', 'Employed', 'Full_time', 'Part_time',
       'Full_time_year_round', 'Unemployed', 'Unemployment_rate', 'Median',
       'P25th', 'P75th', 'College_jobs', 'Non_college_jobs', 'Low_wage_jobs'],
      dtype='object')

In [68]:
df_grad = pd.read_csv('data/grad-students.csv')
df_grad.columns

Index(['Major_code', 'Major', 'Major_category', 'Grad_total',
       'Grad_sample_size', 'Grad_employed', 'Grad_full_time_year_round',
       'Grad_unemployed', 'Grad_unemployment_rate', 'Grad_median', 'Grad_P25',
       'Grad_P75', 'Nongrad_total', 'Nongrad_employed',
       'Nongrad_full_time_year_round', 'Nongrad_unemployed',
       'Nongrad_unemployment_rate', 'Nongrad_median', 'Nongrad_P25',
       'Nongrad_P75', 'Grad_share', 'Grad_premium'],
      dtype='object')

In [69]:
df_stem = pd.read_csv('data/women-stem.csv')
df_stem.columns

Index(['Rank', 'Major_code', 'Major', 'Major_category', 'Total', 'Men',
       'Women', 'ShareWomen', 'Median'],
      dtype='object')

## SQLite3

In [2]:
import sqlite3,csv

In [55]:
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute("CREATE TABLE all_ages (Major_code Int, Major Text, Major_category Text, Total Int, Employed Int, Employed_full_time_year_round Int, Unemployed Int, Unemployment_rate Real,Median Int,P25th Int,P75th Int);")

In [56]:
with open('data/all-ages.csv','r') as fin: 
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Major_code'], i['Major'], i['Major_category'], i['Total'], i['Employed'],i['Employed_full_time_year_round'],i['Unemployed'], i['Unemployment_rate'],i['Median'],i['P25th'],i['P75th']) for i in dr]


In [57]:
cur.executemany("INSERT INTO all_ages (Major_code, Major, Major_category, Total, Employed, Employed_full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th) VALUES (?,?,?,?,?,?,?,?,?,?,?);", to_db)
conn.commit()

In [61]:
# worst non engineering majors
query = '''
select Major, Total, Median, Unemployment_rate
from all_ages
where (Major_category != 'Engineering') and (Unemployment_rate > 0.065)
order by Unemployment_rate desc
limit 20;
'''
#non_engineering = conn.execute(query).fetchall()
conn.execute(query).fetchall()

[('MISCELLANEOUS FINE ARTS', 8511, 45000, 0.156147487),
 ('CLINICAL PSYCHOLOGY', 7638, 45000, 0.102712161),
 ('MILITARY TECHNOLOGIES', 4315, 64000, 0.101796407),
 ('SCHOOL STUDENT COUNSELING', 2396, 41000, 0.101745936),
 ('LIBRARY SCIENCE', 16193, 40000, 0.094842992),
 ('VISUAL AND PERFORMING ARTS', 55141, 40000, 0.094658002),
 ('COMPUTER PROGRAMMING AND DATA PROCESSING', 29317, 60000, 0.090264217),
 ('SOCIAL PSYCHOLOGY', 10871, 47000, 0.087336245),
 ('ASTRONOMY AND ASTROPHYSICS', 4700, 80000, 0.086021505),
 ('FILM VIDEO AND PHOTOGRAPHIC ARTS', 133508, 47000, 0.085618911),
 ('COMMUNICATION TECHNOLOGIES', 62141, 50000, 0.085008669),
 ('STUDIO ARTS', 81008, 37600, 0.083713827),
 ('INDUSTRIAL AND ORGANIZATIONAL PSYCHOLOGY', 17969, 62000, 0.08362907),
 ('MASS MEDIA', 211213, 48000, 0.08300476),
 ('MISCELLANEOUS PSYCHOLOGY', 34102, 45000, 0.082009364),
 ('LINGUISTICS AND COMPARATIVE LANGUAGE AND LITERATURE',
  75791,
  48000,
  0.081348089),
 ('DRAMA AND THEATER ARTS', 174817, 42000, 0.0802

In [71]:
# best majors
query = '''
select Major, Unemployment_rate
from all_ages
where Unemployment_rate < 0.05
order by Unemployment_rate
limit 20;
'''
conn.execute(query).fetchall()

[('EDUCATIONAL ADMINISTRATION AND SUPERVISION', 0.0),
 ('GEOLOGICAL AND GEOPHYSICAL ENGINEERING', 0.0),
 ('PHARMACOLOGY', 0.016110797),
 ('MATERIALS SCIENCE', 0.022333333),
 ('MATHEMATICS AND COMPUTER SCIENCE', 0.024900398),
 ('GENERAL AGRICULTURE', 0.026147106),
 ('TREATMENT THERAPY PROFESSIONS', 0.026291603),
 ('NURSING', 0.026796818),
 ('AGRICULTURE PRODUCTION AND MANAGEMENT', 0.028636059),
 ('AGRICULTURAL ECONOMICS', 0.030248324),
 ('MEDICAL ASSISTING SERVICES', 0.031356845),
 ('PLANT SCIENCE AND AGRONOMY', 0.031790887),
 ('MATHEMATICS TEACHER EDUCATION', 0.032983017),
 ('TEACHER EDUCATION: MULTIPLE LEVELS', 0.033356863),
 ('BOTANY', 0.034023515),
 ('PHARMACY PHARMACEUTICAL SCIENCES AND ADMINISTRATION', 0.034357681),
 ('MEDICAL TECHNOLOGIES TECHNICIANS', 0.036209865),
 ('ATMOSPHERIC SCIENCES AND METEOROLOGY', 0.036726308),
 ('ELEMENTARY EDUCATION', 0.038359164),
 ('PHYSICAL SCIENCES', 0.038952537)]

In [54]:
conn.close()

## MySQL 

In [41]:
# for python 3, install via
# pip install mysqlclient
import MySQLdb

In [75]:
os.chdir('./data')
os.getcwd()

'/home/matej/develop/mkm_notebooks/data'

In [78]:
!cat create_all_ages_indexed.sql

CREATE TABLE all_ages
  ( Major_code INTEGER UNSIGNED,
    Major VARCHAR(80) ,
    Major_category VARCHAR(80),
    Total INTEGER UNSIGNED,
    Employed INTEGER UNSIGNED,
    Employed_full_time_year_round INTEGER UNSIGNED,
    Unemployed INTEGER UNSIGNED,
    Unemployment_rate FLOAT,
    Median INTEGER UNSIGNED,
    P25th INTEGER UNSIGNED,
    P75th INTEGER UNSIGNED,
    CONSTRAINT pk_code PRIMARY KEY(Major_code)
);


In [79]:
!cat load_all_ages.sql

LOAD DATA LOCAL INFILE 'all-ages.csv'
INTO TABLE all_ages
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Major_code, Major, Major_category, Total, Employed, Employed_full_time_year_round, Unemployed, Unemployment_rate, Median, P25th, P75th)


In [77]:
# load in data 
!mysql college_majors < create_all_ages_indexed.sql
!mysql college_majors < load_all_ages.sql

In [51]:
# needs to be explicit
con = MySQLdb.connect(host='localhost', user='matej',passwd='xyz&#!384', db='college_majors');

In [52]:
cur = con.cursor()
cur.execute("SELECT VERSION()")
cur.fetchone()

('10.0.21-MariaDB',)

In [63]:
query = '''
select Major, Total, Median, Unemployment_rate
from all_ages
where (Major_category != 'Engineering') and (Unemployment_rate > 0.065)
order by Unemployment_rate desc
limit 20;
'''
cur = con.cursor()
cur.execute(query)
cur.fetchall()

(('MISCELLANEOUS FINE ARTS', 8511, 45000, 0.156147),
 ('CLINICAL PSYCHOLOGY', 7638, 45000, 0.102712),
 ('MILITARY TECHNOLOGIES', 4315, 64000, 0.101796),
 ('SCHOOL STUDENT COUNSELING', 2396, 41000, 0.101746),
 ('LIBRARY SCIENCE', 16193, 40000, 0.094843),
 ('VISUAL AND PERFORMING ARTS', 55141, 40000, 0.094658),
 ('COMPUTER PROGRAMMING AND DATA PROCESSING', 29317, 60000, 0.0902642),
 ('SOCIAL PSYCHOLOGY', 10871, 47000, 0.0873362),
 ('ASTRONOMY AND ASTROPHYSICS', 4700, 80000, 0.0860215),
 ('FILM VIDEO AND PHOTOGRAPHIC ARTS', 133508, 47000, 0.0856189),
 ('COMMUNICATION TECHNOLOGIES', 62141, 50000, 0.0850087),
 ('STUDIO ARTS', 81008, 37600, 0.0837138),
 ('INDUSTRIAL AND ORGANIZATIONAL PSYCHOLOGY', 17969, 62000, 0.0836291),
 ('MASS MEDIA', 211213, 48000, 0.0830048),
 ('MISCELLANEOUS PSYCHOLOGY', 34102, 45000, 0.0820094),
 ('LINGUISTICS AND COMPARATIVE LANGUAGE AND LITERATURE',
  75791,
  48000,
  0.0813481),
 ('DRAMA AND THEATER ARTS', 174817, 42000, 0.0802737),
 ('PUBLIC POLICY', 14782, 6000

In [72]:
con.close()